In [ ]:
#i "nuget: C:/Users/ncgui/Desktop/PointProcessDecoder/artifacts/package/release"

In [ ]:
#r "nuget: PointProcessDecoder.Core, 0.1.4"
#r "nuget: PointProcessDecoder.Cpu, 0.1.4"
#r "nuget: PointProcessDecoder.Plot, 0.1.4"
#r "nuget: PointProcessDecoder.Simulation, 0.1.4"

In [ ]:
using System;
using System.IO;

using TorchSharp;
using static TorchSharp.torch;

using PointProcessDecoder.Core;
using PointProcessDecoder.Plot;
using PointProcessDecoder.Simulation;
using PointProcessDecoder.Core.Estimation;
using PointProcessDecoder.Core.Transitions;
using PointProcessDecoder.Core.Encoder;
using PointProcessDecoder.Core.Decoder;
using PointProcessDecoder.Core.StateSpace;
using PointProcessDecoder.Core.Likelihood;

In [ ]:
string EncodeToHtml(byte[] data)
{
    return $"<img src=\"data:image/png;base64,{Convert.ToBase64String(data)}\">";
}

static Tensor ReadBinaryFile(
    string binary_file,
    Device device = null,
    ScalarType scalarType = ScalarType.Float32
)
{
    device ??= CPU;
    byte[] fileBytes = File.ReadAllBytes(binary_file);
    int elementCount = fileBytes.Length / sizeof(double);
    double[] doubleArray = new double[elementCount];
    Buffer.BlockCopy(fileBytes, 0, doubleArray, 0, fileBytes.Length);
    Tensor t = tensor(doubleArray, device: device, dtype: scalarType);
    return t;
}

public static (Tensor, Tensor) InitializeRealData(
    string positionFile,
    string marksFile,
    Device device = null,
    ScalarType scalarType = ScalarType.Float32
)
{
    var position = ReadBinaryFile(positionFile, device, scalarType);
    var marks = ReadBinaryFile(marksFile, device, scalarType);
    return (position, marks);
}

Heatmap PlotPrediction(
    Tensor prediction,
    Tensor position2D,
    string title,
    double xMin,
    double xMax,
    double yMin,
    double yMax,
    string outputDirectory
)
{
    Heatmap plotPrediction = new(
        xMin,
        xMax,
        yMin,
        yMax,
        title: title
    );

    plotPrediction.OutputDirectory = Path.Combine(plotPrediction.OutputDirectory, outputDirectory);
    plotPrediction.Show<float>(
        prediction,
        position2D
    );
    plotPrediction.Save(png: true);
    
    return plotPrediction;
}

In [ ]:
string positionFile = "../data/positions_2D.bin";
string marksFile = "../data/marks.bin";

Device device = CPU;
ScalarType scalarType = ScalarType.Float32;
int markDimensions = 4;
int markChannels = 28;

var (position, marks) = InitializeRealData(
    positionFile: positionFile,
    marksFile: marksFile,
    device: device,
    scalarType: scalarType
);

position = position.reshape(-1, 2);
marks = marks.reshape(position.shape[0], markDimensions, markChannels);

In [ ]:
position = position[TensorIndex.Slice(0, 100000)];
marks = marks[TensorIndex.Slice(0, 100000)];

In [ ]:
var pointProcessModel = new PointProcessModel(
    estimationMethod: EstimationMethod.KernelCompression,
    transitionsType: TransitionsType.RandomWalk,
    encoderType: EncoderType.ClusterlessMarkEncoder,
    decoderType: DecoderType.StateSpaceDecoder,
    stateSpaceType: StateSpaceType.DiscreteUniformStateSpace,
    likelihoodType: LikelihoodType.Clusterless,
    minStateSpace: [0, 0],
    maxStateSpace: [120, 120],
    stepsStateSpace: [50, 50],
    observationBandwidth: [2, 2],
    stateSpaceDimensions: 2,
    markDimensions: markDimensions,
    markChannels: markChannels,
    markBandwidth: [1, 1, 1, 1],
    ignoreNoSpikes: true,
    distanceThreshold: 1.5,
    sigmaRandomWalk: 1,
    device: device,
    scalarType: scalarType
);

# Encode

In [ ]:
double fractionTraining = 0.8;
int trainingBatchSize = 10000;
int testingBatchSize = 100;

In [ ]:
int nTraining = (int)(position.shape[0] * fractionTraining);

for (int i = 0; i < nTraining + 1; i += trainingBatchSize)
{
    Console.WriteLine($"Training batch {i / trainingBatchSize + 1} of {nTraining / trainingBatchSize + 1}");
    var end = Math.Min(i + trainingBatchSize, nTraining);
    pointProcessModel.Encode(
        position[TensorIndex.Slice(i, end)],
        marks[TensorIndex.Slice(i, end)]
    );
}

# Decode

In [ ]:
int nTesting = (int)position.shape[0] - nTraining;
for (int i = nTraining; i < nTraining + nTesting + 1; i += testingBatchSize)
{
    Console.WriteLine($"Testing batch {(i - nTraining) / testingBatchSize + 1} of {nTesting / testingBatchSize + 1}"); 
    var end = Math.Min(i + testingBatchSize, nTraining + nTesting);
    var prediction = pointProcessModel.Decode(marks[TensorIndex.Slice(i, end)]);
    prediction = (prediction.sum(dim: 0) / prediction.sum()).reshape([50, 50]);
    var title = $"Prediction2D_{i}-{end}";
    var positionSampled = position[TensorIndex.Slice(i, end)];
    var heatmap = PlotPrediction(
        prediction, 
        positionSampled, 
        title,
        0,
        120,
        0,
        120,
        "20250125_ClusterlessMark2D_b71d4a7"
    );
}

# Encode and Decode

In [ ]:
int nTraining = (int)(position.shape[0] * fractionTraining);
int nTesting = (int)position.shape[0] - nTraining;

for (int i = 0; i < nTraining + nTesting + 1; i += testingBatchSize)
{
    Console.WriteLine($"Testing batch {i / testingBatchSize + 1} of {(nTraining + nTesting) / testingBatchSize + 1}"); 
    var end = Math.Min(i + testingBatchSize, nTraining + nTesting);
    
    if (i < nTraining) {
        pointProcessModel.Encode(
            position[TensorIndex.Slice(i, end)],
            marks[TensorIndex.Slice(i, end)]
        );
    }

    var prediction = pointProcessModel.Decode(marks[TensorIndex.Slice(i, end)]);
    prediction = (prediction.sum(dim: 0) / prediction.sum())
        .reshape(pointProcessModel.StateSpace.Shape);
    var title = $"Prediction2D_{i}-{end}";
    var positionSampled = position[TensorIndex.Slice(i, end)];
    var heatmap = PlotPrediction(
        prediction, 
        positionSampled, 
        title,
        0,
        120,
        0,
        120,
        $"20250125_ClusterlessMark2D_b71d4a7_EncodeAndDecode_NTraining{nTraining}_NTesting{nTesting}"
    );
}